In [3]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

import time
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any
from pathlib import Path
from IPython.display import clear_output
from collections import defaultdict

import os
import dotenv
from qubx import QubxLogConfig
from qubx.utils.runner import run_ccxt_trading

from qubx import lookup, logger, QubxLogConfig
from qubx.core.basics import TriggerEvent, Trade, MarketEvent, Instrument, Subtype, FundingRate
from qubx.core.interfaces import IStrategyContext, IStrategy
from qubx.connectors.ccxt.ccxt_connector import CCXTExchangesConnector
from qubx.connectors.ccxt.ccxt_trading import CCXTTradingConnector
from qubx.utils.runner import get_account_config
from qubx.pandaz import scols
from qubx.backtester.simulator import SimulatedTrading
from qubx.utils.runner import run_ccxt_paper_trading
from qubx.utils.collections import TimeLimitedDeque
from qubx.plotting.interfaces import IPlotter
from qubx.plotting.data import PlotData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
class TradeTestStrat(IStrategy, IPlotter):
    def on_init(self, ctx: IStrategyContext):
        ctx.set_base_subscription(Subtype.OHLC["1m"])
        ctx.subscribe(Subtype.TRADE)
        ctx.set_warmup({
            Subtype.OHLC["1h"]: "20d",
            Subtype.OHLC["1m"]: "2h",
            Subtype.TRADE: "10m",
        })
    
    def get_plots(self) -> list[str]:
        return ["features"]
    
    def get_plot_data(self, plot_name: str) -> PlotData:
        match plot_name:
            case "features":
                self.plot_features()
        raise ValueError(f"Unknown plot name: {plot_name}")
    
    def plot_features(self) -> PlotData:
        pass


ctx = run_ccxt_paper_trading(
    strategy=(stg := TradeTestStrat()),
    exchange="BINANCE.UM",
    symbols=["BTCUSDT", "ETHUSDT", "ADAUSDT", "XRPUSDT"],
    blocking=False,
)